# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
J. Wolf  ->  D. J. Wolf  |  ['J. Wolf']
S. Torniamenti  ->  S. Torniamenti  |  ['S. Torniamenti']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']


Arxiv has 63 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.07932


extracting tarball to tmp_2505.07932...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2505.07932/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'photo_z_info' from 'tmp_2505.07932/photo_z_info.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'sel_photometric_info' from 'tmp_2505.07932/sel_photometric_info.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.

...

Found 132 bibliographic references in tmp_2505.07959/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2505.08002


extracting tarball to tmp_2505.08002...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']


Found 466 bibliographic references in tmp_2505.08002/aa54444-25.bbl.
Issues with the citations
syntax error in line 172: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.07932-b31b1b.svg)](https://arxiv.org/abs/2505.07932) | **Lyman-Break Galaxies in the Mpc-Scale Environments Around Three $z\sim7.5$ Quasars With JWST Imaging**  |
|| M. Pudoka, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>J. Wolf</mark> |
|*Appeared on*| *2025-05-14*|
|*Comments*| *31 Pages, 10 Figures, Accepted by ApJ*|
|**Abstract**|            We study the Mpc-scale environments of the three highest redshift luminous quasars at $z\geq 7.5$ (J031343.84-180636.40, J134208.11+092838.61, and J100758.27+211529.21) to understand their connection to large-scale structure. Cosmological simulations show that these early supermassive black holes (SMBHs) are expected to form in the most massive dark matter halos. Therefore, it is expected that they are anchors of galaxy overdensities if luminous matter traces the underlying dark matter structure of the Universe. Using JWST NIRCam (F090W/F115W/F250M/F360M/F430M) imaging, we observe the large-scale structure out to $\sim13$ comoving Mpc around these quasars. We select F090W-dropout Lyman Break galaxies (LBGs) and F430M-excess [OIII] emitters in the three fields. We find 18, 21, and 6 LBG candidates in the fields of J0313, J1342, and J1007, respectively, resulting in a wide range of overdensities ($1+\delta \sim 19,\,24,$ and $7$). The photometric redshifts indicate serendipitous foreground and background overdensities in the J0313 field. The joint angular autocorrelation of the combined LBG sample shows significant clustering on $<1.8$ comoving Mpc scales, demonstrating that the selected galaxies are likely associated with the large-scale structure surrounding the quasars. This first systematic study of $z\sim 7.5$ quasars shows a diverse set of quasar environments at the onset of their formation, providing empirical data to help constrain theoretical predictions of early structure formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.07959-b31b1b.svg)](https://arxiv.org/abs/2505.07959) | **Impact of stellar winds on the pair-instability supernova rate**  |
|| F. Simonato, et al. -- incl., <mark>S. Torniamenti</mark> |
|*Appeared on*| *2025-05-14*|
|*Comments*| *12 pages, 10 figures, 2 tables. Comments welcome*|
|**Abstract**|            Very massive stars (VMSs, $M_{\star}$ $\geq$ 100 M$_{\odot}$) play a crucial role in several astrophysical processes. At low metallicity, they might collapse directly into black holes, or end their lives as pair-instability supernovae. Recent observational results set an upper limit of $0.7\,{}\mathrm{ yr}^{-1} \,{}\mathrm{ Gpc}^{-3}$ on the rate density of pair-instability supernovae in the nearby Universe. However, most theoretical models predict rates exceeding this limit. Here, we compute new VMS tracks with the MESA code, and use them to analyze the evolution of the (pulsational) pair-instability supernova rate density across cosmic time. We show that stellar wind models accounting for the transition between optically thin and thick winds yield a pair-instability supernova rate $\mathcal{R}_{\mathrm{PISN}}\sim{}0.1$ Gpc$^{-3}$ yr$^{-1}$ at redshift $z\sim{}0$, about two orders of magnitude lower than our previous models. We find that the main contribution to the pair-instability supernova rate comes from stars with metallicity $Z\sim{}0.001-0.002$. Stars with higher metallicities cannot enter the pair-instability supernova regime, even if their zero-age main sequence mass is up to 500 M$_\odot$. The main reason is that VMSs enter the regime for optically thick winds during the main sequence at metallicity as low as $Z\sim{4}\times{}10^{-4}$. This enhances the mass loss rate, quenching the growth of the He core and thus preventing the onset of pair-instability in later evolutionary stages. This result highlights the critical role of mass loss in shaping the final fate of very massive stars and the rate of pair-instability supernovae.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.08002-b31b1b.svg)](https://arxiv.org/abs/2505.08002) | **JWST Observations of Young protoStars (JOYS): overview of program and early results**  |
|| E. v. Dishoeck, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-05-14*|
|*Comments*| *47 pages, 28 + 7 figures, A&A, in press*|
|**Abstract**|            The embedded phase is a crucial period in the development of a young star. Mid-IR observations, now possible with JWST with unprecedented sensitivity, spectral resolution and sharpness are key for probing many physical and chemical processes on sub-arcsecond scales. JOYS addresses a wide variety of questions, from protostellar accretion and the nature of primeval jets, winds and outflows, to the chemistry of gas and ice, and the characteristics of embedded disks. We introduce the program and show representative results. MIRI-MRS data of 17 low-mass and 6 high-mass protostars show a wide variety of features. Atomic line maps differ among refractory (e.g., Fe), semi-refractory (e.g., S) and volatile elements (e.g., Ne), linked to their different levels of depletion and local (shock) conditions. Nested, stratified jet structures consisting of an inner ionized core seen in [Fe II] with an outer H2 layer are commonly seen. Wide-angle winds are found in low-J H2 lines. [S I] follows the jet in the youngest protostars, but is concentrated on source when more evolved. [Ne II] reveals a mix of jet shock and photoionized emission. H I lines measure accretion, but are also associated with jets. Molecular emission (CO2, C2H2, HCN, H2O, ..) is cool compared with disks, and likely associated with hot cores. Deep ice absorption features reveal not just the major ice components but also ions (as part of salts) and complex organic molecules, with comparable abundances from low- to high-mass sources. A second detection of HDO ice in a solar-mass source is presented with HDO/H2O ~ 0.4%, providing a link with disks and comets. A deep search for solid O2 suggests it is not a significant oxygen reservoir. Only few embedded Class I disks show the same forest of water lines as Class II disks do, perhaps due to significant dust extinction of the upper layers [abridged].         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.07932/./cutouts/cutout_J0313_586.png', 'tmp_2505.07932/./cutouts/cutout_J0313_734.png', 'tmp_2505.07932/./cutouts/cutout_J0313_1554.png', 'tmp_2505.07932/./cutouts/cutout_J0313_1555.png', 'tmp_2505.07932/./cutouts/cutout_J0313_2075.png', 'tmp_2505.07932/./ACF_combine_fields_revised.png', 'tmp_2505.07932/./template_dropout.png']
copying  tmp_2505.07932/./cutouts/cutout_J0313_586.png to _build/html/
copying  tmp_2505.07932/./cutouts/cutout_J0313_734.png to _build/html/
copying  tmp_2505.07932/./cutouts/cutout_J0313_1554.png to _build/html/
copying  tmp_2505.07932/./cutouts/cutout_J0313_1555.png to _build/html/
copying  tmp_2505.07932/./cutouts/cutout_J0313_2075.png to _build/html/
copying  tmp_2505.07932/./ACF_combine_fields_revised.png to _build/html/
copying  tmp_2505.07932/./template_dropout.png to _build/html/
exported in  _build/html/2505.07932.md
    + _build/html/tmp_2505.07932/./cutouts/cutout_J0313_586.png
    + _build/html/tmp_2505.07932/./cutouts/cutou

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\rb}[1]{\textbf{\textcolor{red}{#1}}}$</div>



<div id="title">

# Lyman-Break Galaxies in the Mpc-Scale Environments Around Three $z\sim 7.5$ Quasars With JWST Imaging

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.07932-b31b1b.svg)](https://arxiv.org/abs/2505.07932)<mark>Appeared on: 2025-05-14</mark> -  _31 Pages, 10 Figures, Accepted by ApJ_

</div>
<div id="authors">

M. Pudoka, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>J. Wolf</mark>

</div>
<div id="abstract">

**Abstract:** We study the Mpc-scale environments of the three highest redshift luminous quasars at $z\geq 7.5$ (J031343.84-180636.40, J134208.11+092838.61, and J100758.27+211529.21) to understand their connection to large-scale structure.  Cosmological simulations show that these early supermassive black holes (SMBHs) are expected to form in the most massive dark matter halos.  Therefore, it is expected that they are anchors of galaxy overdensities if luminous matter traces the underlying dark matter structure of the Universe. Using JWST NIRCam (F090W/F115W/F250M/F360M/F430M)imaging, we observe the large-scale structure out to $\sim13$ comoving Mpc around these quasars.  We select F090W-dropout Lyman Break galaxies (LBGs) and F430M-excess [ $\ion{O}{3}$ ] emitters in the three fields.  We find 18, 21, and 6LBG candidates in the fields of J0313, J1342, and J1007, respectively,  resulting in a wide range of overdensities ( $1+\delta \sim 19, 24,$ and $7$ ).  The photometric redshifts indicate serendipitous foreground and background overdensities in the J0313 field.  The joint angular autocorrelation of the combined LBG sample shows significant clustering on $<1.8$ comoving Mpc scales, demonstrating that the selected galaxies are likely associated with the large-scale structure surrounding the quasars.  This first systematic study of $z\sim 7.5$ quasars shows a diverse set of quasar environments at the onset of their formation, providing empirical data to help constrain theoretical predictions of early structure formation.

</div>

<div id="div_fig1">

<img src="tmp_2505.07932/./cutouts/cutout_J0313_586.png" alt="Fig4.1" width="20%"/><img src="tmp_2505.07932/./cutouts/cutout_J0313_734.png" alt="Fig4.2" width="20%"/><img src="tmp_2505.07932/./cutouts/cutout_J0313_1554.png" alt="Fig4.3" width="20%"/><img src="tmp_2505.07932/./cutouts/cutout_J0313_1555.png" alt="Fig4.4" width="20%"/><img src="tmp_2505.07932/./cutouts/cutout_J0313_2075.png" alt="Fig4.5" width="20%"/>

**Figure 4. -** Cutouts of each selected LBG candidate. Each panel is a $2\times2$ arcminute cutout in the F090W, F115W, F250M, F360M, and F430M filters from left to right with magnitudes shown in the top left of each panel (a red background means it is not detected and replaced by the $2\sigma$ limit). Crosshairs are to guide the eye and each cutout color is normalized separately to account for the large dynamical range of flux in each filter. (*fig:cut1*)

</div>
<div id="div_fig2">

<img src="tmp_2505.07932/./ACF_combine_fields_revised.png" alt="Fig3" width="100%"/>

**Figure 3. -** The Landy-Szalay estimated angular ACF for the combined fields shown as blue markers. Errors along the x-axis are the bin sizes while y-axis errors show the Poisson errors described in the text.  Open circles show the ACF for randomly distributed sources within the field consistent with zero.  The solid blue line shows the MCMC fit within the constrained bin range excluding the largest and smallest bins ($\beta=1.5$) while the dashed section shows the fit results extrapolated to the full range of the data.  The shaded regions represents the 16th and 84th percentiles of the posterior distribution of $A_{\omega}$.  In magenta is the same best fit keeping $\beta = 0.8$.  (*fig:acfplot*)

</div>
<div id="div_fig3">

<img src="tmp_2505.07932/./template_dropout.png" alt="Fig12" width="100%"/>

**Figure 12. -** _Left:_ Transmission curves of the NIRCam filters used in this analysis are shaded in grey with the  rest-optical spectrum of a galaxy selected from the JAGUAR suite of galaxy templates redshifted to $z=7.5$ is shown in blue. The three black vertical ticks show the H$\beta$ line and [$\ion${O}{3}] doublet from left to right.  In magenta is a representative L-type brown dwarf from the Sonora brown dwarf models with $T_{\rm eff} = 1100$ K and (F090W-F115W, F115W-F250M)$=(2.03, -2.41)$.  _Right:_ Color vs redshift for the JAGUAR galaxy templates from $z=6$ to $z=9$ calculated using the reported F090W and F115W fluxes in the catalog taking into account the image depths.  Using only galaxy templates that are detected in F115W, the blue shaded region shows the 1$\sigma$ spread in color values in bins containing an equal number of galaxies.  The solid line shows the average value in each bin.  The grey region shows the dropout selection criteria of F090W-F115W$>$1.5 while the black dashed line shows F090W-F115W=2 discussed in the text. (*fig:dropout*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.07932"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ST}[1]{\textcolor{steelblue!100}{#1_{\mathrm{ST}}}}$</div>



<div id="title">

# Impact of stellar winds on the pair-instability supernova rate

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.07959-b31b1b.svg)](https://arxiv.org/abs/2505.07959)<mark>Appeared on: 2025-05-14</mark> -  _12 pages, 10 figures, 2 tables. Comments welcome_

</div>
<div id="authors">

F. Simonato, et al. -- incl., <mark>S. Torniamenti</mark>

</div>
<div id="abstract">

**Abstract:** Very massive stars (VMSs, $M_{\star}$ $\geq$ 100 M $_{\odot}$ ) play a crucial role in several astrophysical processes. At low metallicity, they might   collapse directly into black holes, or end their lives as pair-instability supernovae.      Recent observational results set an upper limit of $0.7  \mathrm{ yr}^{-1}   \mathrm{ Gpc}^{-3}$ on the  rate density of pair-instability supernovae in the nearby Universe. However, most theoretical models predict rates  exceeding this limit.      Here, we compute new VMS tracks with the mesa code, and use them to analyze the evolution of the (pulsational) pair-instability supernova rate density across cosmic time.   We show that stellar wind models accounting for the transition between optically thin and  thick winds  yield a pair-instability supernova rate $\mathcal{R}_{\mathrm{PISN}}\sim 0.1$ Gpc $^{-3}$ yr $^{-1}$ at redshift $z\sim 0$ , about two orders of magnitude lower than our previous models. We find that the main contribution to the pair-instability supernova rate comes from stars with metallicity $Z\sim 0.001-0.002$ . Stars with higher metallicities cannot enter the pair-instability supernova regime, even if their zero-age main sequence mass is up to 500 M $_\odot$ . The main reason is that VMSs enter the regime for optically thick winds during the main sequence at metallicity as low as $Z\sim{4}\times 10^{-4}$ . This enhances the mass loss rate, quenching the growth of the He core and thus preventing the onset of pair-instability in later evolutionary stages.   This result highlights the critical role of mass loss  in shaping the final fate of very massive stars and the rate of pair-instability supernovae.

</div>

<div id="div_fig1">

<img src="tmp_2505.07959/./Figure/contour_sv_pc.png" alt="Fig2" width="100%"/>

**Figure 2. -** Contour plot showing the He core masses at the end of the He burning for our mesa tracks adopting \protect\citetalias{Sabhahit2023} winds  as a function of the metallicity and ZAMS mass. The white lines highlight the levels at $32 \protect\text{M}_{\odot}$, $ 64 \protect\text{ M}_{\odot}$ and $135 \protect\text{ M}_{\odot}$, which set the boundaries to have PPISNe, PISNe, and direct collapse via photodisintegration. (*fig:He_mass_cont*)

</div>
<div id="div_fig2">

<img src="tmp_2505.07959/./Figure/pisn_rate_density_new_2.png" alt="Fig9" width="100%"/>

**Figure 9. -** PISN (upper panels) and PPISN (lower panels) rate density evolution as a function of redshift.
    The left-hand and right-hand plots show the results obtained with the new wind model by \protect\citetalias{Sabhahit2023}  and the one by \protect\citetalias{Chen2015}, respectively. The black thick line represents the total rate density, whereas the colored lines show the contribution of individual metallicities from $Z = 1 \times 10^{-4}$(violet solid line) to $Z = 2 \times 10^{-2}$(dashed yellow line). (*fig:pisn_new*)

</div>
<div id="div_fig3">

<img src="tmp_2505.07959/./Figure/contour_sv_pc_old.png" alt="Fig3" width="100%"/>

**Figure 3. -** Same as Fig. \ref{fig:He_mass_cont}, but for models  \protect\citetalias{Chen2015}. (*fig:He_mass_cont_old*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.07959"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.0}$</div>



<div id="title">

# JWST Observations of Young protoStars (JOYS): Overview of program and early results

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.08002-b31b1b.svg)](https://arxiv.org/abs/2505.08002)<mark>Appeared on: 2025-05-14</mark> -  _47 pages, 28 + 7 figures, A&A, in press_

</div>
<div id="authors">

E. v. Dishoeck, et al. -- incl., <mark>H. Beuther</mark>, <mark>C. Gieser</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The embedded phase of star formation is a crucial period in the    development of a young star as the system still accretes matter,    emerges from its natal cloud assisted by powerful jets and    outflows, and forms a disk setting the stage for the birth of a    planetary system. Mid-infrared spectral line observations, now    possible with unprecedented sensitivity, spectral resolution and    sharpness with the $_ James Webb Space Telescope_$ (JWST), are    key for probing many of the physical and chemical processes on    sub-arcsecond scales that occur in highly extincted regions,    providing unique diagnostics and complementing millimeter    observations. The JWST Observations of Young protoStars (JOYS) program aims to    address a wide variety of questions, ranging from protostellar    accretion and the nature of primeval jets, winds and outflows, to    the chemistry of gas and ice in hot cores and cold dense    protostellar environments, and the characteristics of the embedded    disks. We introduce the program and show representative JOYS    results. JWST Mid-InfraRed Instrument (MIRI) Medium Resolution Spectrometer    (MRS) Integral Field Unit (IFU) 5--28 $\mu$ m maps of 17 low-mass    targets (23 if binary components counted individually) and 6    high-mass protostellar sources are taken with resolving powers $R=\lambda/\Delta \lambda=1500-4000$ . Small mosaics ranging from $1\times 1$ to $3\times 3$ MRS tiles cover $\sim 4"$ to $20"$ fields of view, providing spectral imaging on spatial scales down    to $\sim$ 30 au (low mass) and $\sim$ 600 au (high mass). For HH    211, the complete $\sim 1'$ blue outflow lobe has been mapped with    the MRS.  Atomic lines are interpreted with published shock    models, whereas molecular lines are analyzed with simple rotation    diagrams and LTE slab models. The importance of taking infrared    pumping into account is stressed. Inferred abundance ratios are    compared with detailed hot core chemical models including    X-rays. Ice spectra are fitted through comparison with laboratory    spectra. The JWST MIRI-MRS spectra show a wide variety of features, with    their spatial distribution providing insight into their physical    origin. Atomic line maps differ among refractory (e.g., Fe),      semi-refractory (e.g., S) and volatile elements (e.g., Ne), and      are linked to their different levels of depletion and local      (shock) conditions. Jets are prominently seen in lines of [ Fe    II ] and other refractory elements whereas the pure rotational    $H_2$ lines probe hot ( $\sim 1000$ K) and warm (few $\times 10^2$ K) gas inside the cavity, associated with jets, outflows and    cavity walls, for both low- and high-mass sources.  Wide-angle    winds are found in low- $J$ $H_2$ lines. Nested, stratified jet    structures consisting of an inner ionized core with an outer    molecular layer are commonly seen in the youngest sources. [ S      I ] follows the jet as seen in [ Fe II ] in the youngest      protostars, but is different in more evolved sources where it is      concentrated on source.  Noble gas lines such as [ Ne II ] 12.8 $\mu$ m reveal a mix of jet shock and photoionized emission.  H I    recombination lines serve as a measure of protostellar accretion    rates, but are also associated with more extended jets. Gaseous    molecular emission ($CO_2$ , $C_2$ $H_2$ , HCN, $H_2$ O, $CH_4$ ,    $SO_2$ , SiO) is seen toward several sources, but is cool compared    with what is found in more evolved disks, with excitation    temperatures of only 100--250 K, and likely associated with the    warm inner envelopes ("hot cores") . $CO_2$ is often extended    along the outflow, in contrast with $C_2$ $H_2$ which is usually    centered on source. Water emission is commonly detected on source    even if relatively weak; off source it is seen only in the highest    density shocks such as associated with NGC 1333 IRAS4B. Some    sources show gaseous molecular lines in absorption, including    $NH_3$ in one case. Deep ice features are seen toward the    protostars, revealing not just the major ice components but also    ions (as part of salts) and complex organic molecules, with    comparable abundances from low- to high-mass sources. Relative    abundances of some gas and ice species are similar, consistent    with ice sublimation in hot cores. A second detection of HDO ice    in a solar-mass source is presented, with an HDO/$H_2$ O ice ratio    of $\sim$ 0.4 \% , providing a link with HDO/$H_2$ O in disks and    comets. A deep search for solid $O_2$ suggests that it is not a    significant oxygen reservoir. Only few embedded Class I disks show    the same forest of water lines as Class II disks do, possibly    caused by significant dust extinction of the upper layers due    to limited growth and settling of dust to the midplane in young    disks as well as radial drift bringing in small dust. This paper illustrates the many different science questions that a    single MIRI-MRS IFU data set can address, with significant    similarities between low- and high-mass sources. Large source    samples across evolutionary stages and luminosities are needed to    further develop these diagnostics of the physics and chemistry of    protostellar systems.

</div>

<div id="div_fig1">

<img src="tmp_2505.08002/./JOYS_Fig2a.png" alt="Fig29.1" width="50%"/><img src="tmp_2505.08002/./JOYS_Fig2b.png" alt="Fig29.2" width="50%"/>

**Figure 29. -** JWST MIRI-MRS spectrum of the low-mass Class 0 protostars
  B1-c (top) and Serpens SMM3 (bottom) illustrating the different
  molecular gas (red) and ice (blue) features that can be observed and
  analyzed, tracing different components of the protostellar
  system. The inserts in the B1-c spectrum aim to highlight the
  gas-phase molecular lines in this source. Atomic emission lines are
  present but not very strong in this source. In
  contrast, both $H_2$ and atomic emission lines are very prominent
  for Serpens SMM3 (bottom). At the bottom of the silicate feature at
  10 $\mu$m, close to the noise limit, the spectra have been binned by
  a factor of 4 to enhance the $S/N$. See Fig. B.3 and B.11 of
  \citet{vanGelder24overview} for individual MIRI-MRS sub-bands. (*fig:B1c-overview*)

</div>
<div id="div_fig2">

<img src="tmp_2505.08002/./JOYS_Fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Cartoon of a protostellar source with the various physical
  components studied in this work indicated. (*fig:protostar-cartoon*)

</div>
<div id="div_fig3">

<img src="tmp_2505.08002/./JOYS_Fig10.png" alt="Fig8" width="100%"/>

**Figure 8. -** MIRI-MRS maps of the various $H_2$ lines toward the Class 0
  protostar Serpens SMM3. The white contours in the top left panel
  indicate the dust disk seen in millimeter continuum. The red and
  blue circles on the S(4) image indicate the positions where the
  $H_2$ spectra and rotational diagrams have been extracted. Note that
  the S(3) image at 9.66 $\mu$m is strongly affected by silicate
  extinction, and the S(5) line at 6.9 $\mu$m by ice extinction, hence
  their maps are more noisy. Beam sizes are indicated in the lower
  left corner of each panel. Each panel is scaled to the maximum
  emission of that species. The maximum and minimum colors are (in Jy
  km s$^{-1}$), from top left to bottom right: S(8) (0.008, 0.003);
  S(7) (0.033, 0.007); S(6) (0.013, 0.003); S(5) (0.026, 0.003); S(4)
  (0.089, 0.009); S(3) (0.035, 0.005); S(2) (0.090, 0.009); S(1)
  (0.084, 0.008).  (*fig:SMM3H2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.08002"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

488  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
